# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 20 2022 10:10AM,253477,10,7365926,"Nextsource Biotechnology, LLC",Released
1,Dec 20 2022 10:03AM,253476,10,WT5479 Out,SugarBear,Released
2,Dec 20 2022 9:34AM,253471,18,INNOVA-8926192,Innova Softgel LLC,Released
3,Dec 20 2022 9:16AM,253470,10,CTF0011555,"Citieffe, Inc.",Released
4,Dec 20 2022 9:05AM,253468,12,HH8553,Hush Hush,Executing
5,Dec 20 2022 8:59AM,253460,10,0086257411,ISDIN Corporation,Released
6,Dec 20 2022 8:59AM,253460,10,0086257410,ISDIN Corporation,Released
7,Dec 20 2022 8:59AM,253460,10,0086257424,ISDIN Corporation,Released
8,Dec 20 2022 8:59AM,253460,10,0086257422,ISDIN Corporation,Released
9,Dec 20 2022 8:59AM,253460,10,0086257423,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,253468,Executing,1
32,253470,Released,1
33,253471,Released,1
34,253476,Released,1
35,253477,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253468,1.0,NaN
253470,NaN,1.0
253471,NaN,1.0
253476,NaN,1.0
253477,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253377,0.0,1.0
253382,1.0,0.0
253383,1.0,0.0
253391,0.0,10.0
253394,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253377,0,1
253382,1,0
253383,1,0
253391,0,10
253394,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253377,0,1
1,253382,1,0
2,253383,1,0
3,253391,0,10
4,253394,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253377,,1
1,253382,1,
2,253383,1,
3,253391,,10
4,253394,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 20 2022 10:10AM,253477,10,"Nextsource Biotechnology, LLC"
1,Dec 20 2022 10:03AM,253476,10,SugarBear
2,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC
3,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc."
4,Dec 20 2022 9:05AM,253468,12,Hush Hush
5,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation
43,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC"
45,Dec 20 2022 8:46AM,253459,10,"CLINUVEL, Inc."
47,Dec 20 2022 8:40AM,253457,16,"Emersa Waterbox, LLC"
48,Dec 20 2022 8:40AM,253455,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 20 2022 10:10AM,253477,10,"Nextsource Biotechnology, LLC",,1
1,Dec 20 2022 10:03AM,253476,10,SugarBear,,1
2,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,,1
3,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",,1
4,Dec 20 2022 9:05AM,253468,12,Hush Hush,1,
5,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,,38
6,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC",,2
7,Dec 20 2022 8:46AM,253459,10,"CLINUVEL, Inc.",,2
8,Dec 20 2022 8:40AM,253457,16,"Emersa Waterbox, LLC",,1
9,Dec 20 2022 8:40AM,253455,10,ISDIN Corporation,,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 20 2022 10:10AM,253477,10,"Nextsource Biotechnology, LLC",1,
1,Dec 20 2022 10:03AM,253476,10,SugarBear,1,
2,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,1,
3,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",1,
4,Dec 20 2022 9:05AM,253468,12,Hush Hush,,1
5,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,38,
6,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC",2,
7,Dec 20 2022 8:46AM,253459,10,"CLINUVEL, Inc.",2,
8,Dec 20 2022 8:40AM,253457,16,"Emersa Waterbox, LLC",1,
9,Dec 20 2022 8:40AM,253455,10,ISDIN Corporation,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 20 2022 10:10AM,253477,10,"Nextsource Biotechnology, LLC",1,
1,Dec 20 2022 10:03AM,253476,10,SugarBear,1,
2,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,1,
3,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",1,
4,Dec 20 2022 9:05AM,253468,12,Hush Hush,,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 20 2022 10:10AM,253477,10,"Nextsource Biotechnology, LLC",1.0,NaN
1,Dec 20 2022 10:03AM,253476,10,SugarBear,1.0,NaN
2,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,1.0,NaN
3,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",1.0,NaN
4,Dec 20 2022 9:05AM,253468,12,Hush Hush,NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 20 2022 10:10AM,253477,10,"Nextsource Biotechnology, LLC",1.0,0.0
1,Dec 20 2022 10:03AM,253476,10,SugarBear,1.0,0.0
2,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,1.0,0.0
3,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",1.0,0.0
4,Dec 20 2022 9:05AM,253468,12,Hush Hush,0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3294839,88.0,2.0
12,253468,0.0,1.0
15,1774010,54.0,2.0
16,253457,1.0,0.0
18,253471,1.0,0.0
19,1013728,54.0,1.0
21,1013590,2.0,2.0
22,253444,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3294839,88.0,2.0
1,12,253468,0.0,1.0
2,15,1774010,54.0,2.0
3,16,253457,1.0,0.0
4,18,253471,1.0,0.0
5,19,1013728,54.0,1.0
6,21,1013590,2.0,2.0
7,22,253444,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,88.0,2.0
1,12,0.0,1.0
2,15,54.0,2.0
3,16,1.0,0.0
4,18,1.0,0.0
5,19,54.0,1.0
6,21,2.0,2.0
7,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,12,Released,0.0
2,15,Released,54.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21,22
Status,,,,,,,,
Executing,2.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0
Released,88.0,0.0,54.0,1.0,1.0,54.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21,22
0,Executing,2.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0
1,Released,88.0,0.0,54.0,1.0,1.0,54.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21,22
0,Executing,2.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0
1,Released,88.0,0.0,54.0,1.0,1.0,54.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()